# AGDC-V2 API for WOfS  (unification merged)

Last modified & tested OK on 2016-06-21 

Fei Zhang

##  Outstanding Issues & Questions


### Bug: list_cells() the platforms can be 1 or 3 or more, but cannot be two satellites like 
qdict={'latitude': (-36.0, -35.0), 'platform': ['LANDSAT_5', 'LANDSAT_7'], 'longitude': (149.01, 150.1), 'time': ('1990-01-01', '2016-03-31')}

### gw constructor argument product repeated in function?

nbar_tiles = gw.list_tiles(product='ls7_nbar_albers')

### Why not keep the function list_tiles_by_cells( cell_list)??

### load nbar-pqa tile data over a cell at a timestamp

nbar_data = gw.load(cell, tile)  # is cell really necessary?? not really. Modified to new load(tile)

In [ ]:
##  Setting up environment: python modules etc

# * Optional! 
# * Skip this section if you use other means to ensure correct env setup, 
# * such as module load before starting jupyter notebook.

# make my current source code python modules available in this notebook
import sys

paths=sys.path
#paths.append('/g/data1/u46/fxz547/Githubz/agdc-v2')
paths.insert(0,'/g/data1/u46/fxz547/Githubz/agdc-v2')  #prepend a path
paths.append('/g/data1/u46/fxz547/Githubz/wofs')


In [ ]:
from IPython.display import display
from pprint import pprint
from __future__ import print_function

from collections import defaultdict
import xarray as xr
import xarray.ufuncs

import datacube
from datacube.api import GridWorkflow
from datacube.storage import  masking
#from datacube.index import index_connect
#from datacube.config import LocalConfig
#from datacube.api._conversion import to_datetime
#new updated version: from datacube.api import make_mask, describe_flags

import numpy as np
import math

#from pyproj import Proj, transform
from osgeo import gdal, osr, ogr
from rasterio.warp import reproject, RESAMPLING
import rasterio

%matplotlib inline
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10 # increase plot size a bit...
rcParams['axes.formatter.useoffset'] = False  # disable scalar formatter / offset in axes labels


Python envionment and modules are ready to go

## Datacube Grid Workflow APIs

In [ ]:
dc = datacube.Datacube( app='wofs-dev')
#or to use a specific config file: 
# dc = datacube.Datacube(config='/home/547/adh547/unification.datacube.conf', app='wofs-dev')

In [ ]:

my_product = 'ls5_nbar_albers' 

gw = GridWorkflow(dc.index , product=my_product)

## List_cells ()
There is now no need to call list_cells for tile-based workflows, unless you just want to list the area covered:

In [ ]:
# product_type= nbar | pqa
qdict={'latitude': (-36.0, -35.0), 'platform': ['LANDSAT_5', 'LANDSAT_7', 'LANDSAT_8'], 'longitude': (149.01, 150.1), 'time': ('1990-01-01', '2016-03-31')}

cells = gw.list_cells(product_type='nbar', product= my_product, **qdict)
                      #longitude=(149.06,149.18), latitude=(-35.27, -35.33), time=('1996-01-01', '2016-03-20'))
len(cells)

In [ ]:
# cells is now is huge dict, like the following
# defaultdict(dict,
#             {(15, -41): {'geobox': <datacube.model.GeoBox at 0x7fc05f3e3390>,
#               'sources': <xarray.DataArray (time: 111)>
#               array([ (Dataset <id=6ce1955f-5ef9-4876-8bb8-69638f45efe8 type=ls5_nbar_albers location=/g/data/u46/users/gxr547/unicube/LS5_TM_NBAR/LS5_TM_NBAR_3577_15_-41_19900302231139000000.nc>,),
#                      (Dataset <id=94efe23b-f699-4a16-a244-ab0fc6b58461 type=ls5_nbar_albers location=/g/data/u46/users/gxr547/unicube/LS5_TM_NBAR/LS5_TM_NBAR_3577_15_-41_19900403231042000000.nc>,),
#                      (Dataset <id=e0f376e7-a7db-4cbb-b137-b9a2865c2b02 type=ls5_nbar_albers location=/g/data/u46/users/gxr547/unicube/LS5_TM_NBAR/LS5_TM_NBAR_3577_15_-41_19900708231050000000.nc>,),
#                      (Dataset <id=8eb3a1af-e906-419e-a22e-d523a65fa830 type=ls5_nbar_albers location=/g/data/u46/users/gxr547/unicube/LS5_TM_NBAR/LS5_TM_NBAR_3577_15_-41_19900724231046000000.nc>,),
#                      (Dataset <id=7c317cb9-b8de-48b8-92a5

In [ ]:
# To get the cell indexes
cells.keys()

# How to get satellite and sensor from the datacube?

In [ ]:
prodlist=dc.list_products()  #pandas df

prodlist[prodlist.name=='ls5_nbar_albers'] # the row for your product


In [ ]:
prodlist


In [ ]:
# get your values as numpy.ndarr
pppl=prodlist[prodlist.name=='ls5_nbar_albers'][['platform','instrument']].values

In [ ]:
type(pppl)

In [ ]:
pppl.shape

In [ ]:
satname=pppl[0,0]
satsensor=pppl[0,1]
print(satname, satsensor)

## List_Tiles()

In [ ]:
nbar_tiles = gw.list_tiles(product_type='nbar', product=my_product, platform=['LANDSAT_5', 'LANDSAT_7', 'LANDSAT_8'],
                           longitude=(149.06,149.18), latitude=(-35.27, -35.33), 
                           time=('1996-01-01', '2016-03-20'))

sorted(nbar_tiles.keys())

In [ ]:
# This nbar_tiles includes scenes and tiles, a very very long list
# nbar_tiles has 18 cells as keys
len(nbar_tiles)

When using the search terms, there is not currently a way to filter out non-tile based data, such as scenes, which go far beyond the requested area. For now we must use **`product='ls5_nbar_albers'`**.

In [ ]:
#bug if just 2 platforms in the list?
#qdict={'latitude': (-36.0, -35.0), 'longitude': (149.01, 150.1),'platform':['LANDSAT_5', 'LANDSAT_7'], 'time': ('1990-01-01', '2016-03-31')}
qdict={'latitude': (-36.0, -35.0), 'longitude': (149.01, 150.1),
       'platform':['LANDSAT_5', 'LANDSAT_7','LANDSAT_8'], 
       'time': ('1990-01-01', '2016-03-31')}

print (qdict)
nbar_tiles = gw.list_tiles(product=my_product,**qdict)
                       
sorted_keylist = sorted(nbar_tiles.keys())

print (len(sorted_keylist))

sorted_keylist[:5]

In [ ]:
nbar_tiles.items()[:1]

In [ ]:
# Pixel Quality Tiles
qdict={'latitude': (-36.0, -35.0), 'longitude': (149.01, 150.1),
       'platform':['LANDSAT_5', 'LANDSAT_7','LANDSAT_8'], 
       'time': ('1990-01-01', '2016-03-31')}
print (qdict)
pq_tiles = gw.list_tiles(product= 'ls5_pq_albers',**qdict)
         
sorted_keys=sorted(pq_tiles.keys())
print(len(sorted_keys))
sorted_keys[:5]

In [ ]:
# No PQ data for Canberra region (15, -40)??

In [ ]:
pq_tiles.items()[0]

In [ ]:
# after unification:
import logging 
def get_nbarpqa_tiles_by_cell( acell, qdict): # refactor the param product='ls5_nbar_albers' OR 'ls5_pq_albers'
    """
    return a list of tiles
    :param acell: a cell index tuple (15, -40)
    :return:
    """
    # gw.list_tiles((15,-40), product='ls5_nbar_albers')

    nbar_tiles = gw.list_tiles(acell, product='ls5_nbar_albers',
                                    **qdict)  # , platform='LANDSAT_8')  # ,time=('2000', '2007'))
    pq_tiles = gw.list_tiles(acell, product='ls5_pq_albers',
                                  **qdict)  # , platform='LANDSAT_8')  # , time=('2000', '2007'))

    if (len(pq_tiles) == len(nbar_tiles)):
        print("The cells have %s nbar and %s pq tiles", len(nbar_tiles), len(pq_tiles))
    else:
        logging.warn("Mismatched NBAR-PQA tiles: The cells have %s nbar and %s pq tiles", len(nbar_tiles),
                     len(pq_tiles))

    # Cell, Time -> Product -> TileDef
    tile_def = defaultdict(dict)

    for index, tile in nbar_tiles.items():
        tile_def[index[:2], index[2]]['nbar'] = tile

    for index, tile in pq_tiles.items():
        tile_def[index[:2], index[2]]['pqa'] = tile


    for index, products in tile_def.items():
        if len(products) < 2:
            logging.warn('un-paired nbar-pqa product for cell %s', str(index))
            logging.warn("remove this un-paired tile from the dict")
            tile_def.pop(index)
        else:
            logging.debug('%s,%s', index, len(products))

    return tile_def



Get the key for the first tile:

In [ ]:
#Get the key for the tile:
acell=(15,-40)
qdict = {'platform': ['LANDSAT_5'], 'time': ('1980-01-01', '2016-03-31')}

tile_def=get_nbarpqa_tiles_by_cell( (15,-40), qdict )
#keys = list(tile_def)

In [ ]:

tile_keys=tile_def.keys()

print (len(tile_keys))

for key in tile_keys[:10]:
    print (key)
    cell = key[0]
    dt_stamp=key[1]
    
    print (dt_stamp)
    
    #ISO 
    isostr=str(dt_stamp)[:19].replace(':','-')      
    print (isostr)



In [ ]:
key= tile_keys[0]
tile_def[key]

In [ ]:
key

# Load the nbar data for a given tile (as keyed by cell and time)

In [ ]:
time_slice=1

cell=(15,-40)

key= tile_keys[time_slice]
print(key)

tile = tile_def[key]['nbar']

print (cell)
print (tile)

nbar_data = gw.load(tile) 

nbar_data

In [ ]:
# It looks that nbar_data.blue is 3D array (time,y,x); But actually you cannot do nbar_data.blue[1, :,:]??
type(nbar_data.blue)
nbar_data.blue

In [ ]:
y_size=4000
x_size=4000

#del raw_image
raw_image = np.zeros((6, y_size, x_size), dtype='int16') #'float32')

raw_image[0,:,:] = nbar_data.blue[:,:]
raw_image[1,:,:] = nbar_data.green[:,:]
raw_image[2,:,:] = nbar_data.red[:,:]
raw_image[3,:,:] = nbar_data.nir[:,:]
raw_image[4,:,:] = nbar_data.swir1[:,:]
raw_image[5,:,:] = nbar_data.swir2[:,:]

In [ ]:
raw_image.shape

In [ ]:
type(raw_image)

In [ ]:
from wofs.waters.detree.classifier import WaterClassifier

classifier = WaterClassifier()

# TODO: water classification using the input data tiles

water_classified_img = classifier.classify(raw_image)



In [ ]:
# display all or part of the classified image
plt.imshow(water_classified_img )  #show Lake BurleyGriffin  clfimg[2200:2600,1500:2500]) if cell=(15,-40)
plt.colorbar(orientation='vertical', shrink=0.8, label='water=128 or no water=0');

In [ ]:
# The first step produce a very rough water tile as shown above. It even has water in no data region.
# This is why it must be filtered by no_data, cloud, etc. below


# NBAR data

We can use the `nodata` value, or use the contiguous bit in the PQ data to mask out nulls.

Here is `nodata`:

In [ ]:
no_data = (nbar_data.green == nbar_data.green.nodata)
no_data.plot()

In [ ]:
type(no_data)

In [ ]:
# a mask 1+2D array
no_data

In [ ]:
nbar_data.green.where(~no_data).plot()

In [ ]:
nbar_data.green.where(~ no_data[0]).plot()

In [ ]:
# Get the nodata values for each band (in case they aren't the same)
no_data_values = nbar_data.apply(lambda data_array: data_array.nodata).to_array(dim='band')

# Turn the Dataset into a DataArray, so we can check all bands
stack = nbar_data.to_array(dim='band')

# Find all values that are set to no data, from any band
no_data_mask = (stack == no_data_values).any(dim='band')

no_data_mask.plot()


In [ ]:
no_data_mask

In [ ]:
no_data_mask.squeeze()

## Load the PQ data:

In [ ]:
# cell = key[0]
tile = tile_def[key]['pqa']
pq_data = gw.load( tile)

We can look at the bits available to us:

In [ ]:
masking.get_flags_def(pq_data)

In [ ]:
# better display using pandas
import pandas
pandas.DataFrame.from_dict(masking.get_flags_def(pq_data), orient='index')

We can make a mask from a combination of bits. Here we will use the cloud masks and the contiguous bit.

In [ ]:
contiguous = masking.make_mask(pq_data, contiguous=1).pixelquality
contiguous.plot()

In [ ]:
no_contiguous = masking.make_mask(pq_data, contiguous=0).pixelquality
no_contiguous.plot()

In [ ]:
# are the same?
np.sum (contiguous != no_contiguous)

In [ ]:
good_data = masking.make_mask(pq_data, cloud_acca='no_cloud', cloud_fmask='no_cloud', contiguous=True).pixelquality
good_data.plot()

In [ ]:
good_data  # is a mask image True/false


In [ ]:
cloud_free_nbar = nbar_data.where(good_data)
cloud_free_nbar.red.plot()

In [ ]:
cloud_free_nbar.red

# How to mask out the 100% GOOD pixels from NBAR data?

In [ ]:
pq_data

In [ ]:
perfect_flag = int(masking.get_flags_def(pq_data)['ga_good_pixel']['values'].keys()[0])   # should be 16383
# perfect_flag = 16383

print ('perfect_flag', perfect_flag)

perfect_pixel_mask=(pq_data.pixelquality == perfect_flag)
perfect_pixel_mask.plot()

In [ ]:
perfect_nbar_data = nbar_data.where(perfect_pixel_mask)


In [ ]:
y_size=4000
x_size=4000

#del raw_image
raw_image = np.zeros((6, y_size, x_size), dtype='int16') #'float32')

raw_image[0,:,:] = perfect_nbar_data.blue[:,:]
raw_image[1,:,:] = perfect_nbar_data.green[:,:]
raw_image[2,:,:] = perfect_nbar_data.red[:,:]
raw_image[3,:,:] = perfect_nbar_data.nir[:,:]
raw_image[4,:,:] = perfect_nbar_data.swir1[:,:]
raw_image[5,:,:] = perfect_nbar_data.swir2[:,:]


water_img = classifier.classify(raw_image)

In [ ]:
# display all or part of the classified image
#plt.imshow(water_img)
plt.imshow(water_img[2200:2600,1500:2500])  #show Lake BurleyGriffin  clfimg[2200:2600,1500:2500]) if cell=(15,-40)
plt.colorbar(orientation='vertical', shrink=0.8, label='water=128 or no water=0');

In [ ]:
water_img

In [ ]:
pq_data

In [ ]:
perfect_pixel_mask = masking.make_mask(pq_data, ga_good_pixel=True).pixelquality

In [ ]:
perfect_pixel_mask = masking.make_mask(pq_data.pixelquality, ga_good_pixel=True)


In [ ]:
perfect_pixel_mask.plot()

# How to get DSM tiles

In [ ]:
acellindex=(15,-40)
dsm_cells = gw.list_tiles(acellindex, product='dsm1sv10')
print (dsm_cells)


In [ ]:
# ----------------------------------------------------------------
def get_dsm_data(acellindex, qdict={}):
    """
    get dsm data for sia, terrainshadow, highslope filters
    :param acellindex:
    :param qdict:
    :return:
    """
    query = qdict.copy()  #avoid modify the original input qdict in-situ
    query.pop('time', None)
    query.pop('platform', None)
    dsm_cells = gw.list_cells(acellindex, product='dsm1sv10', **query)

    def load_dsm(cell):
        data = gw.load(cell)
        return data.squeeze('time').drop('time')

    return {index: load_dsm(cell) for index, cell in dsm_cells.items()}


In [ ]:
dsmtile= get_dsm_data(acellindex, qdict={})

In [ ]:
print(dsmtile)

In [ ]:

dsmtile[(15,-40)]['elevation'].plot()